# How to access the Temporal Memory algorithm directly

This program demonstrates how to create a TM instance, train it, get predictions and
anomalies, and inspect the state.

The code here runs a very simple version of sequence learning, with one cell per column. The TM is trained with the simple sequence A->B->C->D->E

In [ ]:
from htm.bindings.sdr import SDR
from htm.algorithms import TemporalMemory as TM

## Aux functions

Utility routine for printing an SDR in a particular way.

In [ ]:
def formatBits(sdr):
    s = ''
    for c in range(sdr.size):
        if c > 0 and c % 10 == 0:
            s += ' '
        s += str(sdr.dense.flatten()[c])
    s += ' '
    return s

def printStateTM( tm ):
    # Useful for tracing internal states
    print("Active cells         " + formatBits(tm.getActiveCells()))
    print("Winner cells         " + formatBits(tm.getWinnerCells()))
    tm.activateDendrites(True)
    print("Predictive cells     " + formatBits(tm.getPredictiveCells()))
    print("Anomaly", tm.anomaly * 100, "%")
    print("")

## Creating the Temporal Memory

In [ ]:
tm = TM(
    columnDimensions = (50,),
    cellsPerColumn=1,
    initialPermanence=0.5,
    connectedPermanence=0.5,
    minThreshold=8,
    maxNewSynapseCount=20,
    permanenceIncrement=0.1,
    permanenceDecrement=0.0,
    activationThreshold=8,
)
tm.printParameters()

__My edit__. Optional playing around w/ SDR: cached property, in-place update

In [ ]:
x = SDR(10)
print("before: ", x.dense, "; or sparse: ", x.sparse)
x.dense[3] = 1
print("after: ", x.dense, "; or sparse: ", x.sparse, " ==> changes aren't visible in sparse, because we haven't notified about them")
x.dense[3] = 1
x.dense = x.dense
print("after: ", x.dense, "; or sparse: ", x.sparse, " ==> set operation notifies about them, so now they're visible")
print("That's because magical properties (dense, sparse, coordinates) are just a cache and can be out of sync with the data they should represent")
print("Accessing another property invalidates cache! So you should always notify about changes before accessing another magical property")

## Creating inputs to feed to the TM

Each input is an SDR representing the active mini-columns.

Here we create a simple sequence of 5 SDRs representing the sequence A -> B -> C -> D -> E

In [ ]:
dataset = { inp : SDR( tm.numberOfColumns() ) for inp in "ABCDE" }
dataset['A'].dense[0:10]  = 1     # Input SDR representing "A", corresponding to mini-columns 0-9
dataset['B'].dense[10:20] = 1     # Input SDR representing "B", corresponding to mini-columns 10-19
dataset['C'].dense[20:30] = 1     # Input SDR representing "C", corresponding to mini-columns 20-29
dataset['D'].dense[30:40] = 1     # Input SDR representing "D", corresponding to mini-columns 30-39
dataset['E'].dense[40:50] = 1     # Input SDR representing "E", corresponding to mini-columns 40-49

# Notify the SDR object that we've updated its dense data in-place.
for z in dataset.values():
    z.dense = z.dense

for inp in "ABCDE":
    print("Input:", inp, " Bits:", formatBits( dataset[inp]) )

## Send this simple sequence to the temporal memory for learning.

The compute method performs one step of learning and/or inference.

__Note__: here we just perform learning but you can perform prediction/inference and learning in the same step if you want (online learning).

In [ ]:
for inp in "ABCDE": # Send each letter in the sequence in order
    print("Input:", inp)
    activeColumns = dataset[inp]

    print(">>> tm.compute()")
    tm.compute(activeColumns, learn = True)

    printStateTM(tm)

## Reset command

The reset command tells the TM that a sequence just ended and essentially zeros out all the states. It is not strictly necessary but it's a bit messier without resets, and the TM learns quicker with resets.

In [ ]:
print(">>> tm.reset()")
tm.reset()

## Resend the same sequence and look at predictions made by TM

The following prints out the active cells, predictive cells, active segments and winner cells.

What you should notice is that the mini-columns where active state is 1 represent the SDR for the current input pattern and the columns where predicted state is 1 represent the SDR for the next expected pattern.

In [ ]:
for inp in "ABCDE":
    print("Input:", inp)
    activeColumns = dataset[inp]

    print(">>> tm.compute()")
    tm.compute(activeColumns, learn = False)

    printStateTM(tm)